# OWA Anholt Array Efficiency: Benchmark Evaluation Script
*Javier Sanz Rodrigo, Fernando Borbón, Pedro Miguel Fernandes Correia, Pawel Gancarski (CENER)* 

*February 2019*

## Introduction
This is the model evaluation script for the [OWA-Anholt Array Efficiency benchmark](https://thewindvaneblog.com/the-owa-anholt-array-efficiency-benchmark-436fc538597d) as part of the [OWA Wake Modeling Challenge](https://www.carbontrust.com/media/677495/owa-wake-modelling-challenge_final-feb27.pdf).



## Simulations
The following models participate in the benchmark.

**Table 1. Participating models.**

| simID | Model | Type | Remarks |  
|:----:|:-----:|:----------:|:--------:|
| anh01 | XXX | time series  | - | 
| anh02 | XXX | time series  | - | 
| anh03 | XXX | time series  | - | 
| anh04 | XXX | time series  | - | 
| anh05 | XXX | time series  | - | 
| anh06 | XXX | time series  | - | 
| anh07 | XXX | bin averages | - | 

In [ ]:
sim_id =    ['anh01','anh03','anh04','anh05','anh06','anh07']
sim_model = ['ENG'  ,'RANS' ,'LES'  ,'RANS' ,'XXX'  ,'XXX'  ]
sim_type =  ['ts'   ,'ts'   ,'ts'   ,'ts'   ,'ts'   ,'ba'   ]

## Load libraries

In [ ]:
%matplotlib inline
from src.WindConditions import *
from src.BinAvrg import *  
from scipy import interpolate

## Wind farm input data 
The scada_flags dataframe indicates which timestamps have been filtered out in the quality-control process of the SCADA data. This allows to perform the evaluation on validation dataset which only contains situations where the wind farm is operating in normal conditions.

In [ ]:
# setup
datefrom = time_stamp(2013,1,1,0,0,0)    # evaluation period
dateto = time_stamp(2015,6,30,23,0,0)    # evaluation period

siteID = 'Anholt'
Hhub = 81.6         # hub-height
Drot = 120          # rotor diameter
lat_ref = 56.6      # degrees N 
lon_ref = 11.2      # degrees E

# Load manufacturer's power curve 
pwr_curve_file = pd.read_csv('./inputs/Anholt_pwc.csv', sep = ';')
pwr_curve_file['power'] = pwr_curve_file['power']/1000 # scale to MW
pwr_curve = interpolate.interp1d(pwr_curve_file['U'].values.flatten(),pwr_curve_file['power'].values.flatten(), 
                                bounds_error = False, fill_value = 0)
# when converting power to speed use the rev_pwr_curve
###rev_pwr_curve = interpolate.interp1d(pwr_curve_file['power'].values.flatten(),pwr_curve_file['U'].values.flatten())

# Load wind farm layout data
turbines = pd.read_csv("inputs/Anholt_layout.csv")
plot_wf_layout(turbines['X coordinate'], turbines['Y coordinate'],labels = turbines['VDC ID'])

## Read data availability flags
This file is provided to the participants so they can compute their bin-averages using the same timestamps of the validation data.

$min\_data\_availability$ sets the minimum availability of observational data (for 90%, no more than 10% is reconstructed)

$scada\_ts$ list of time stamps that can be analysed

In [ ]:
scada_flags = pd.read_csv('./inputs/obs_flags.csv', index_col = 'time') 
min_data_availability = 90 # minimum data availability in %
scada_ts = flags_to_ts(scada_flags, min_data_availability) # generate a list of accepted time series

## Wind conditions from mesoscale input data
In the absence of an *undisturbed* met mast, the wind farm centroid is used as reference site to define wind conditions and classify the wind climate in terms of wind direction sectors and stability classes. This site has also been used as the center of the innermost domain in the WRF set-up.

In [ ]:
# Define bins to classify wind conditions 
Sbins = np.array([8,10])              # around the maximum of the trust coefficient 
WDbins = np.arange(-15.,360.+15.,30)  # wind direction bins (12 sectors)
WDbins_label = ['N','NNE','ENE','E','ESE','SSE',
                'S','SSW','WSW','W','WNW','NNW']
zLbins = [-0.2,-0.02, 0.02, 0.2]      # 3 stability bins
zLbins_label = ['u','n','s']

# interpolate to reference height
zref = Hhub         # [m]

# Load mesoscale data from the referent site (time-height profiles)
mast = WindConditions('./inputs/Anholt_Lav30km_ref.nc',lat_ref, lon_ref, siteID, datefrom, dateto)

#init the bin averaging class
bin_avrg = BinAvrg(siteID,datefrom, dateto, WDbins, WDbins_label, zLbins, zLbins_label,turbines['VDC ID'],sim_id)

#filter out speed restrictions
scada_ts = bin_avrg.filter_s(mast, zref, scada_ts, Sbins)

# Compute and plot distributions 
N_WDzL_all,_,_,_,_ = mast.plot_stability(WDbins,WDbins_label,zLbins,zLbins_label,zref)

# after filtering
mast.reduce_to_ts(scada_ts)
N_WDzL_speed,_,_,_,_ = mast.plot_stability(WDbins,WDbins_label,zLbins,zLbins_label,zref)

Effect of wind speed restriction on sample size 

In [ ]:
print(N_WDzL_all.sum().sum())
print(N_WDzL_speed.sum().sum())

Apply power curve to mesoscale data

In [ ]:
# Load mesoscale data at turbine positions (time-series at hub-height)
f = netCDF4.Dataset('./inputs/Anholt_WindTurbines.nc', 'r')
meso_ts_windspeed = pd.DataFrame(
            (f.variables['U'][:].data**2 + f.variables['V'][:].data**2)**0.5, 
            index = f.variables['Times'][:].data)
meso_ts_windspeed = restrict_to_ts(meso_ts_windspeed, scada_ts)

meso_ts = meso_ts_windspeed.transform(pwr_curve) # convert to power

Create mapping of time stamps per wind direction and stability bins

In [ ]:
#create mapping
ts_bin_map = bin_avrg.create_ts_to_bin_map(mast, zref, scada_ts)

# Compute bin-averaged (and std) quantities
meso_p, meso_p_std = bin_avrg.compute_mean(meso_ts, ts_bin_map)

## Load simulation data

In [ ]:
# Loop through the submitted simulation data files and categorize them according to Table 1
n_sim = len(sim_id)
sim_ts = []
sim_P = bin_avrg.array_init(('sim', 'wt','wd','zL'))
sim_P_std = bin_avrg.array_init(('sim', 'wt','wd','zL'))

for isim in range(0,n_sim):
    file_name = './outputs/'+ sim_id[isim] +'.csv'
    ts = p = p_std = None
    if sim_type[isim] == 'ts': 
        ts = pd.read_csv(file_name, index_col = 'time')  # read .csv output files
        p, p_std = bin_avrg.compute_mean(ts, ts_bin_map)   # from time-series to bin-averaged quantities
        # clean up the time series data and apply the scada_ts filter
        ts = restrict_to_ts(ts, scada_ts)
    else:
        p = bin_avrg.read_ba_file(file_name)
    sim_ts.append(ts)
    sim_P[isim] = p
    sim_P_std[isim] = p_std
    
# where sim[isim] is a 3D array: 
# Turbine # weind direction bin # stability bin  

## Load validation data
To increase the data availability, a machine learning algorithm has been applied to recover data from turbines that are not working in nominal conditions from others that do.

In [ ]:
try:
    ts_obs = pd.read_csv('./observations/obs.csv', index_col = 'time') 
    p_obs, p_obs_std = bin_avrg.compute_mean(ts_obs, ts_bin_map)  
    # clean up the time series data and apply the scada_ts filter
    ts_obs = restrict_to_ts(ts_obs, scada_ts)
    val_data = True
except IOError:
    print ("No validation data available")
    val_data = False

If no validation data is available, do code to code comparison

In [ ]:
ref_sim = 0

if val_data:
    p_ref = p_obs
    p_ref_std = p_obs_std
else: 
    p_ref = sim_P[ref_sim]
    p_ref_std = sim_P_std[ref_sim]

## Compute quantities of interest and metrics
The *array efficiency* is defined based on the background wind speed at each turbine position:

$$ \eta = \frac{\sum_{i}P_i}{\sum_{i}P(S_i)} $$

where $P_i$ is the power of turbine $i$, observed or simulated, and $P(S_i)$ is the theoretical power from the manufaturer's power curve computed at the background (mesoscale) wind speed $S_i$ at each turbine position.  

Performance is measured in terms of the BIAS:

$$ BIAS = \eta_{obs} - \eta_{sim} $$ 

for each bin and for the whole wind distribution.

In [ ]:
#init the arrays
eta_sim = bin_avrg.array_init(('sim', 'wd','zL'))
bias = bin_avrg.array_init(('sim', 'wd','zL'))
dif_std = bin_avrg.array_init(('sim', 'wd','zL'))
bias_tot = bin_avrg.array_init(('sim',))
mean_abs_error = bin_avrg.array_init(('sim',))
len_array = np.vectorize(len)
bin_sizes = len_array(ts_bin_map)
sum_bin_sizes = bin_sizes.sum()
meso_p_sum = np.sum(meso_p,axis=0) 

#compute variables of interest
eta_ref = np.sum(p_ref,axis=0) / meso_p_sum * 100 # axis=0 are turbines

dif_std_ref = np.mean(p_ref_std - meso_p_std,axis=0)
for isim in range(n_sim):
    eta_sim[isim] = np.sum(sim_P[isim],axis=0) / meso_p_sum * 100
    bias[isim] = eta_ref - eta_sim[isim]
    if sim_P_std[isim] is not None:
        dif_std[isim] = np.mean(sim_P_std[isim] - p_ref_std,axis=0)

    bias_tot[isim] = (bias[isim]*bin_sizes).sum()/sum_bin_sizes
    mean_abs_error[isim] = (np.absolute(bias[isim])*bin_sizes).sum()/sum_bin_sizes

## Plot results

Plot $\eta_{sim}$ and associated $BIAS$ versus wind direcction and stability bins

In [ ]:
figcaption = "**Fig 2. BIAS**"
title = "_BIAS[%]"
bin_avrg.plot_heatmaps(bias, sub_plt_size = (1.7,4),n_plot_cols = 6, 
                       figcaption = figcaption, title=title)

Compute stability related bias

In [ ]:
(np.sum(bias,axis=1)/len(WDbins)).to_pandas()

Compute efficiency and bias for each turbine

In [ ]:
wt_eta_sim = bin_avrg.array_init(('sim', 'wt','wd','zL'))
wt_bias = bin_avrg.array_init(('sim', 'wt','wd','zL'))

wt_eta_ref = p_ref / meso_p * 100
wt_eta_ref_max = (p_ref+p_ref_std/2) / meso_p * 100

for isim in range(n_sim):
    wt_eta_sim[isim] = sim_P[isim] / meso_p * 100
    wt_bias[isim] = wt_eta_ref - wt_eta_sim[isim]

Plot a transect, for chosen bin and wind turbines list

In [ ]:
# Transect: Choose bin (WD,zL), define list of turbines and plot profiles of array efficiency along the transect of turbines. 
#           Normalized by dividing with eta of the first turbine in the list
#           Use error bars showind one std about the observed array efficiency (eta_obs_std)

#for list of turbines and wd, 2D plot Value per distance in rotor diameter. 
data =   wt_eta_sim.loc[:,:,'NNW','u'].to_pandas()
ref_data = wt_eta_ref.loc[:,'NNW','u'].to_pandas()
ref_data_max = wt_eta_ref_max.loc[:,'NNW','n'].to_pandas()

#from P28 to P19 
wt_list = ['ANHP'+str(28-i) for i in range(10)]

plot_transect(data,ref_data,wt_list,turbines,Drot)

In [ ]:
data = wt_eta_sim.loc['anh05',:,'SSE','u'].to_pandas()
plot_wf_layout(turbines['X coordinate'], turbines['Y coordinate'],data = data,vmin=40, vmax=120)

In [ ]:
data = wt_bias.loc['anh05',:,'SSE','u'].to_pandas()
plot_wf_layout(turbines['X coordinate'], turbines['Y coordinate'],data = data)